<h1> Parameter Tunning with MinMax
    
    GridSearchCV for: hidden layers with previous parameters

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# Import data; set X and y; fill nan values and split in test and training  data:

In [2]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# filter max values for offer if required
print(data.Offers.max()) #max is 2500... no need to filter max values

# 2017 & 2018 data
data = data.loc[data.index > 2018060000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

# Fill nan values (BEFORE OR AFTER TEST, TRAIN SPLIT!!!)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X.fillna(X.mean(), inplace = True)
y.fillna(y.mean(), inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size=0.1, shuffle=False)

2500.0


/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


# Apply feature scaling:

In [3]:
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

# Importing the Keras libraries and packages:

In [4]:
import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
import keras.optimizers
from keras import initializers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

Using TensorFlow backend.


# Create regressor:

In [5]:
# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden, n_neurons = 30, optimizer = 'RMSprop', kernel_initializer="he_normal",
    bias_initializer= initializers.Ones()):
    model = Sequential()
    model.add(Dense(output_dim = n_neurons, 
                    input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dense(output_dim = 1, 
                    activation = 'linear'))
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

# Apply Randomized tunning for the whole ANN:

In [6]:
regressor = KerasRegressor(build_fn = regressor_tunning)

# Dictionary to include the parameters
parameters = {'n_hidden': [1, 2, 3, 4, 5, 6]
               }

tscv = TimeSeriesSplit(n_splits = 6)

# add some early stopping
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='mse', patience = 15)

rnd_search_cv = GridSearchCV(estimator = regressor,
                                   param_grid = parameters,
                                   scoring = 'neg_mean_squared_error',
                                   n_jobs = -1,
                                   cv = tscv)

# checkpoints:
# early_stopping_cb = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)
# checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5")

rnd_search_cv.fit(X_train, y_train, batch_size = 20, epochs = 100, callbacks=[early_stopping])


/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=15, units=30)`
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python-cpu/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="linear", units=1)`
  from ipykernel import kernelapp as app


Epoch 1/100
9246/9246 [==============================] - 2s 164us/step - loss: 4777.0344 - mse: 4777.0312 - mae: 47.6443
Epoch 2/100
9246/9246 [==============================] - 1s 96us/step - loss: 1838.7513 - mse: 1838.7515 - mae: 25.5932
Epoch 3/100
9246/9246 [==============================] - 1s 100us/step - loss: 1821.0258 - mse: 1821.0264 - mae: 25.4586
Epoch 4/100
9246/9246 [==============================] - 1s 93us/step - loss: 1818.1903 - mse: 1818.1923 - mae: 25.3388
Epoch 5/100
9246/9246 [==============================] - 1s 95us/step - loss: 1809.1127 - mse: 1809.1124 - mae: 25.3181
Epoch 6/100
9246/9246 [==============================] - 1s 94us/step - loss: 1801.5916 - mse: 1801.5907 - mae: 25.2337
Epoch 7/100
9246/9246 [==============================] - 1s 97us/step - loss: 1798.8296 - mse: 1798.8297 - mae: 25.1654
Epoch 8/100
9246/9246 [==============================] - 1s 96us/step - loss: 1796.8371 - mse: 1796.8373 - mae: 25.1555
Epoch 9/100
9246/9246 [===============

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=6),
             estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7fba6b842950>,
             n_jobs=-1, param_grid={'n_hidden': [1, 2, 3, 4, 5, 6]},
             scoring='neg_mean_squared_error')

# with batch of 32 and 100 epochs:

In [12]:
best_params_1 = rnd_search_cv.best_params_
best_score_1 = rnd_search_cv.best_score_

print("the best parameters are:{}".format(best_params_1))
print("the best score is:{}".format(best_score_1))

the best parameters are:{'n_hidden': 2}
the best score is:-1954.5657673868488


In [11]:
results_cv = rnd_search_cv.cv_results_
print(results)

{'mean_fit_time': array([37.21530068, 35.11886628, 50.00599865, 57.52494407, 62.28967369,
       55.27675267]), 'std_fit_time': array([16.47540748, 12.8374057 , 27.55469417, 30.69671894, 27.02199113,
       18.2099514 ]), 'mean_score_time': array([0.09623369, 0.10681558, 0.12273228, 0.14138965, 0.15728744,
       0.14577214]), 'std_score_time': array([0.0050506 , 0.00678777, 0.00617301, 0.00900352, 0.01967463,
       0.02759429]), 'param_n_hidden': masked_array(data=[1, 2, 3, 4, 5, 6],
             mask=[False, False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_hidden': 1}, {'n_hidden': 2}, {'n_hidden': 3}, {'n_hidden': 4}, {'n_hidden': 5}, {'n_hidden': 6}], 'split0_test_score': array([-1122.49696713, -1119.84323883, -1149.43128488, -1143.12986462,
       -1156.02197316, -1110.42217023]), 'split1_test_score': array([-1775.84807139, -1782.27676771, -1771.8059906 , -1844.70528206,
       -1760.96782862, -2164.94725847]), 'split2_test_scor

# with batch of 20 and 100 epochs:

In [13]:
best_params_1 = rnd_search_cv.best_params_
best_score_1 = rnd_search_cv.best_score_

print("the best parameters are:{}".format(best_params_1))
print("the best score is:{}".format(best_score_1))

the best parameters are:{'n_hidden': 2}
the best score is:-1954.5657673868488
